# Ranking policy training

The tree nodes in MCTS are expanded by an expansion function approximated by a policy graph neural network. The policy network is composed of two parts: molecular representation and reaction rule prediction parts. In the representation part, the molecular graph is converted to a single vector by graph convolutional layers. The training set structure and the prediction part architecture depend on the type of policy network, particularly the ranking or filtering policy network. The training dataset for ranking policy network consists of pairs of reactions and corresponding reaction rules extracted from it. The products of the reaction are transformed to the CGR encoded as a molecular graph with the one-hot encoded label vector where the positive label corresponds to the reaction rule. The prediction part is terminated with the softmax function generating the “probability of successful application” of each reaction rule to a given input molecular graph, which can be used for the reaction rules “ranking”.
All the reaction rules predicted by the ranking or policy neural network are sorted by predicted reaction rule probability and the first N (usually N = 50) of reaction rules are selected to be applied to the current precursor in the expansion step.

**Training set.** The training dataset for ranking policy network consists of pairs of reactions and corresponding reaction rules extracted from it. The products of the reaction are transformed to the CGR encoded as a molecular graph with the one-hot encoded label vector where the positive label corresponds to the reaction rule. The prediction part is terminated with the softmax function generating the “probability of successful application” of each reaction rule to a given input molecular graph, which can be used for the reaction rules “ranking”.

Frst, we define the training configuration using the `PolicyNetworkConfig` class.

## 1. Set up input and output data locations

The input data will be downloaded from the [HuggingFace repository](https://huggingface.co/Laboratoire-De-Chemoinformatique/SynPlanner) to the current directory. 

In [1]:
import os
import pickle
import shutil
from pathlib import Path
from synplan.utils.loading import download_all_data

# download SynPlanner data
data_folder = Path("synplan_data").resolve()
download_all_data(save_to=data_folder)

# results folder
results_folder = Path("tutorial_results").resolve()
results_folder.mkdir(exist_ok=True)

# input data
# use default filtered data from tutorial folder or replace with custom data prepared with data curation tutorial
# be sure that you use the same reaction dataset from which the reaction rules were extracted 
filtered_data_path = results_folder.joinpath("uspto_filtered.smi")
reaction_rules_path = results_folder.joinpath("uspto_reaction_rules.pickle")

# output_data
ranking_policy_network_folder = results_folder.joinpath("ranking_policy_network")
ranking_policy_dataset_path = ranking_policy_network_folder.joinpath("ranking_policy_dataset.pt") # the generated training set

Fetching 25 files:   0%|          | 0/25 [00:00<?, ?it/s]

### 1. Configuration
**Configuration parameters**:

| Parameter        | Default | Description                                                     |
|------------------|---------|-----------------------------------------------------------------|
| vector_dim       | 512     | The dimension of the hidden layers                              |
| num_conv_layers  | 5       | The number of convolutional layers                              |
| learning_rate    | 0.0005  | The learning rate                                               |
| dropout          | 0.4     | The dropout value                                               |
| num_epoch        | 100     | The number of training epochs                                   |
| batch_size       | 1000    | The size of the training batch of input molecular graphs        |

The ranking or filtering policy network architecture and training hyperparameters can be adjusted in the training configuration yaml file below.

In [2]:
from synplan.utils.config import PolicyNetworkConfig
from synplan.ml.training.supervised import create_policy_dataset, run_policy_training

In [3]:
training_config = PolicyNetworkConfig(
    policy_type="ranking",  # the type of policy network
    num_conv_layers=5,  # the number of graph convolutional layers in the network
    vector_dim=512,  # the dimensionality of the final embedding vector
    learning_rate=0.0008,  # the learning rate for the training process
    dropout=0.4,  # the dropout rate
    num_epoch=100,  # the number of epochs for training
    batch_size=100,
)  # the size of training batch of input data

### 2. Creating the training set

Next, we create the policy dataset using the `create_policy_dataset` function. This involves specifying paths to the reaction rules and the reaction data:

In [5]:
datamodule = create_policy_dataset(
    dataset_type="ranking",
    reaction_rules_path=reaction_rules_path,
    molecules_or_reactions_path=filtered_data_path,
    output_path=ranking_policy_dataset_path,
    batch_size=training_config.batch_size,
    num_cpus=4,
)

Number of reactions processed: 69445 [09:36]


Training set size: 52507, validation set size: 13127


### 3. Run the policy network training

Finally, we train the policy network using the `run_policy_training` function. This step involves feeding the dataset and the training configuration into the network:

In [6]:
run_policy_training(
    datamodule,  # the prepared data module for training
    config=training_config,  # the training configuration
    results_path=ranking_policy_network_folder,
)  # path to save the training results

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Policy network balanced accuracy: 0.985
